In [1]:
import numpy as np
from bs4 import BeautifulSoup
from scraperFunctions import vtiScraper,vxusScraper, bndScraper, bndxScraper
import pandas as pd
from selenium import webdriver
import datetime
import time
etfs = ['vti','vxus','bnd','bndx','vtip']

In [ ]:
# Running scrapers (If you already have scraped historical data, go to updater)
vti_MC_data = vtiScraper()
vxus_MC_data = vxusScraper()
bnd_MC_data = bndScraper()
bndx_MC_data = bndxScraper()
# Saving data
vti_MC_data.to_csv('vti_MC_data.csv',index=False)
vxus_MC_data.to_csv('vxus_MC_data.csv',index=False)
bnd_MC_data.to_csv('bnd_MC_data.csv',index=False)
bndx_MC_data.to_csv('bndx_MC_data.csv',index=False)

In [2]:
# Updating data (If already have updated historical data, go to reader)

In [3]:
# Reading saved data
vti_MC_data = pd.read_csv('vti_MC_Data.csv')
vxus_MC_data = pd.read_csv('vxus_MC_Data.csv')
bnd_MC_data = pd.read_csv('bnd_MC_Data.csv')
bndx_MC_data = pd.read_csv('bndx_MC_Data.csv')

In [4]:
# Getting historical price data for etfs (This only takes a couple seconds to run.)
etf_price_data = pd.DataFrame(columns=['Date','ETF','Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume'])
start_epoch = int(datetime.datetime(2000,1,1).timestamp())
curr_epoch = int(time.time())
for etf in etfs:
    url = 'https://query1.finance.yahoo.com/v7/finance/download/'+etf+'?period1='+str(start_epoch)+'&period2='+str(curr_epoch)+'&interval=1d&events=history'
    try:
        df = pd.read_csv(url,parse_dates=[0])
    except ex.HTTPError:
        # Including some breaks to avoid being blocked
        time.sleep(20)
        continue
    df['ETF'] = etf.upper()
    df['AdjClose'] = df['Adj Close'].astype('float64')
    df = df.drop(['Adj Close'],axis=1)
    df = df.dropna(subset=['AdjClose', 'Volume'])
    df['Open']=df['Open'].astype('float64')
    df['High']=df['High'].astype('float64')
    df['Low']=df['Low'].astype('float64')
    df['Close']=df['Close'].astype('float64')
    df['Volume']=df['Volume'].astype('int64')
    etf_price_data = etf_price_data.append(df)
    etf_price_data = etf_price_data.reset_index(drop=True)
# Saving historical price data
etf_price_data.to_csv('etf_price_data.csv',index=False)

In [6]:
# Getting current prices
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome('/Users/avinaashkoganti/chromedriver',options=options)
prices = []
for etf in etfs:
    driver.get('https://finance.yahoo.com/quote/'+etf)
    html = driver.page_source
    soup = BeautifulSoup(html,features="html.parser")
    x = soup.find('span', {'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})
    price = float(x.text)
    prices.append(price)
    print(etf.upper(),price)
driver.close()
prices2 = prices[0:4]

VTI 139.36
VXUS 43.9
BND 87.48
BNDX 56.57
VTIP 49.54


In [ ]:
# format [U.S. Stocks, International Stocks, U.S. Bonds, International Bonds] (in trillions for market cap)
# As of December 30,2019
EOQmarketcaps = np.array([31.780092,22.5263,25.734560,19.55274])
EOQprices = np.array([163.62,83.86,55.69,56.58])
currentPrices = np.asarray(prices2)
amountInvested = 5925

sum(EOQmarketcaps)

In [ ]:
EOQproportions = EOQmarketcaps/sum(EOQmarketcaps)
EOQproportions

In [ ]:
priceRatios = currentPrices / EOQprices
priceRatios

In [ ]:
products = EOQproportions * priceRatios
products

In [ ]:
# Weights for each etf theoretical
revisedProportions = products / sum(products)
revisedProportions

In [ ]:
sum(revisedProportions)

In [ ]:
desiredValues = revisedProportions * amountInvested
desiredValues

In [ ]:
sum(desiredValues)

In [ ]:
# Shares of etfs to buy theoretically
shares = desiredValues/currentPrices
shares

In [ ]:
schwab_shares = shares * .984
schwab_shares

In [ ]:
# Shares of etfs to buy without fractional shares
s = np.round(schwab_shares)
s

In [ ]:
s*currentPrices

In [ ]:
sum(s*currentPrices)

In [ ]:
y = (s*currentPrices)/sum(s*currentPrices)
# array([0.32650404, 0.22523272, 0.25267487, 0.19558837])

In [ ]:
# New weights of portfolio
y * 100

In [ ]:
z = y * np.array([0.03,0.035,0.09,0.09])

In [ ]:
cost = sum(z)/4
cost

In [ ]:
# How much it costs to construct the portfolio
amountInvested * cost/100